In [11]:


# Uninstall mkl for faster neural-network training time
# Upgrade pip to ensure the latest package versions are available
!pip install mxnet --upgrade
!pip install dask --upgrade
# !pip install -U "mxnet<2.0.0"
!pip install autogluon
!pip install -U ipykernel

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
np.random.seed(111)
from __future__ import print_function


import mxnet as mx
import mxnet.ndarray as nd
from mxnet import nd, autograd, gluon
from mxnet.gluon.data.vision import transforms
from autogluon import TextPrediction as task

from src import load_text

ctx = mx.gpu() if mx.context.num_gpus() else mx.cpu()

  Using cached mxnet-1.7.0.post1-py2.py3-none-win_amd64.whl (33.0 MB)
  Using cached numpy-1.16.6.zip (5.1 MB)
  Using cached requests-2.18.4-py2.py3-none-any.whl (88 kB)
  Using cached urllib3-1.22-py2.py3-none-any.whl (132 kB)
  Running setup.py clean for numpy
Failed to build numpy
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.3
    Uninstalling urllib3-1.26.3:
      Successfully uninstalled urllib3-1.26.3
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


  ERROR: Command errored out with exit status 1:
   command: 'c:\users\caell\new_anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\caell\\AppData\\Local\\Temp\\pip-install-rg58yztv\\numpy_8e5bbd8a9932478ea962068d49fc89f1\\setup.py'"'"'; __file__='"'"'C:\\Users\\caell\\AppData\\Local\\Temp\\pip-install-rg58yztv\\numpy_8e5bbd8a9932478ea962068d49fc89f1\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\caell\AppData\Local\Temp\pip-wheel-oh27e96x'
       cwd: C:\Users\caell\AppData\Local\Temp\pip-install-rg58yztv\numpy_8e5bbd8a9932478ea962068d49fc89f1\
  Complete output (376 lines):
  Running from numpy source directory.
  C:\Users\caell\AppData\Local\Temp\pip-install-rg58yztv\numpy_8e5bbd8a9932478ea962068d49fc89f1\numpy\distutils\misc_util.py:476: SyntaxWarning: "is" with a literal. Did you mean "==

  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
    Running setup.py install for numpy: started
    Running setup.py install for numpy: finished with status 'error'
  Rolling back uninstall of numpy
  Moving to c:\users\caell\new_anaconda3\lib\site-packages\numpy
   from C:\Users\caell\new_Anaconda3\Lib\site-packages\~~mpy
  Moving to c:\users\caell\new_anaconda3\lib\site-packages\numpy-1.18.5-py3.8.egg-info
   from C:\Users\caell\new_Anaconda3\Lib\site-packages\~-mpy-1.18.5-py3.8.egg-info
  Moving to c:\users\caell\new_anaconda3\scripts\f2py-script.py
   from C:\Users\caell\AppData\Local\Temp\pip-uninstall-643r3kfl\f2py-script.py
  Moving to c:\users\caell\new_anaconda3\scripts\f2py.exe
   from C:\Users\caell\AppData\Local\Temp\pip-uninstall-643r3kfl\f2py.exe
  Using cached dask-2021.1.1-py3-none-any.whl (891 kB)
  Using cached dask-2021.1.0-py3-none-any.whl (889 kB)


You should consider upgrading via the 'c:\users\caell\new_anaconda3\python.exe -m pip install --upgrade pip' command.


  Using cached urllib3-1.26.3-py2.py3-none-any.whl (137 kB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.22
    Uninstalling urllib3-1.22:
      Successfully uninstalled urllib3-1.22
  Attempting uninstall: requests
    Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Successfully uninstalled requests-2.18.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pmdarima 1.7.1 requires Cython<0.29.18,>=0.29, but you have cython 0.29.21 which is incompatible.
pmdarima 1.7.1 requires setuptools<50.0.0, but you have setuptools 52.0.0 which is incompatible.
gensim 3.8.3 requires Cython==0.29.14, but you have cython 0.29.21 which is incompatible.
You should consider upgrading via the 'c:\users\caell\new_anaconda3\python.exe -m pip install --upgrade pip' command.


  Using cached ipykernel-5.4.3-py3-none-any.whl (120 kB)
  Using cached ipykernel-5.4.2-py3-none-any.whl (119 kB)


You should consider upgrading via the 'c:\users\caell\new_anaconda3\python.exe -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'mxnet'

In [10]:
df = load_text(sentences=True).dropna()


NameError: name 'load_text' is not defined

In [8]:

predictor = task.fit(train_df[['Text','Grade']], label='Grade',
                     seed=111
                    )

NameError: name 'task' is not defined

In [ ]:
randindx = np.random.permutation(np.arange(len(df)))
train_df = df.iloc[randindx[:int(len(randindx)*.8)]]
test_df = df.iloc[randindx[len(train_df):]]
errors = test_df.copy()


ERROR: distributed 2021.1.1 has requirement dask>=2020.12.0, but you'll have dask 2.12.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: botocore 1.19.59 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
I

In [ ]:

predictor = task.fit(train_df[['Text','Grade']], label='Grade',
                     seed=111
                    )
print(predictor.evaluate(test_df[['Text','Grade']], metrics=['mae']))
yhat = predictor.predict({'Text':[text for text in test_df.Text]})
errors['Text Prediction'] = yhat
errors

In [ ]:
predictor2 = task.fit(train_df[['grammarized','Grade']], label='Grade',
                     seed=111
                    )
print(predictor2.evaluate(test_df[['grammarized','Grade']], metrics=['mae']))
yhat = predictor2.predict({'grammarized':[text for text in test_df.grammarized]})
errors['Grammarized Prediction'] = yhat
errors

In [ ]:
predictor3 = task.fit(train_df[['lemmatized','Grade']], label='Grade',
                     seed=111
                    )
print(predictor3.evaluate(test_df[['lemmatized','Grade']], metrics=['mae']))
yhat = predictor3.predict({'lemmatized':[text for text in test_df.Text]})
errors['Lemmatized Prediction'] = yhat
errors

In [ ]:
errors = errors.iloc[:,[1,2,3,0,4,5]]
errors

In [ ]:
predictor.export('TextModel')

In [ ]:
net.export("lenet", epoch=1)


In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    deserialized_net = gluon.nn.SymbolBlock.imports("lenet-symbol.json", ['data'], "lenet-0001.params", ctx=ctx)
